In [17]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [18]:
id_train_features = '1LnKdehYxXYXIUhxO1NFeu5GebmKpbYjY'
id_test_features = '1CIuprbEUeJDEdg0WO5Sq2-xV8OBElXLX'
id_train_labels = '1dzgH430IP8VV4Jw6InbfTiMEFUxZk1pO'
id_test_labels = '10dwdJ5EI_w-HfeM3F8CyctlqAnuW3RY9'

In [19]:
downloaded = drive.CreateFile({'id':id_train_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_features.csv')  
# Dataset is now stored in a Pandas Dataframe
train_X = pd.read_csv('train_features.csv',header=None)
train_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.087,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072
1,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228
2,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123
3,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019
4,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019,0.123


In [20]:
downloaded = drive.CreateFile({'id':id_test_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_features.csv')  
# Dataset is now stored in a Pandas Dataframe
test_X = pd.read_csv('test_features.csv',header=None)
test_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236,0.082


In [21]:
downloaded = drive.CreateFile({'id':id_train_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
train_Y = pd.read_csv('train_labels.csv',header=None)
train_Y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [28]:
downloaded = drive.CreateFile({'id':id_test_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
test_Y = pd.read_csv('test_labels.csv',header=None)
test_Y.head()

,0,1,2,3,4,5
0,c,l,a,s,s,2
1,c,l,a,s,s,2
2,c,l,a,s,s,2
3,c,l,a,s,s,2
4,c,l,a,s,s,2


In [37]:
test_Y = test_Y[5]

In [ ]:
train_y = np.array(train_Y)
test_y = np.array(test_Y)
train_x = list()
train_x.append(train_X)
train_x = np.dstack(train_x)
test_x = list()
test_x.append(test_X)
test_x = np.dstack(test_x)
print('Shape of the train features tensor',train_x.shape)
print('Shape of the train labels tensor',train_y.shape)
print('Shape of the test features tensor',test_x.shape)
print('Shape of the test labels tensor',test_y.shape)

In [50]:
# zero-offset class values
train_y = train_y - 1
test_y = test_y - 1
# one hot encode y
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [68]:
print('Shape of the test labels tensor',test_y.shape)
print('Shape of the train labels tensor',train_y.shape)

Shape of the test labels tensor (258829, 2)
Shape of the train labels tensor (10452, 2)


In [62]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
print(n_timesteps,n_features,n_outputs)

In [66]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 51,102
Trainable params: 51,102
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
verbose, epochs, batch_size = 0, 100, 64
# fit network
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [70]:
# evaluate model on train set
_, accuracy = model.evaluate(train_x, train_y, batch_size=batch_size, verbose=0)
print(accuracy)

0.6220818758010864


In [75]:
# evaluate model on test set
_, accuracy = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
print(accuracy)

0.7560513019561768


In [83]:
print('Shape of the train features tensor',train_x.shape)
print('Shape of the train labels tensor',train_y.shape)
print('Shape of the test features tensor',test_x.shape)
print('Shape of the test labels tensor',test_y.shape)

Shape of the train features tensor (10452, 41, 1)
Shape of the train labels tensor (10452, 2)
Shape of the test features tensor (258829, 41, 1)
Shape of the test labels tensor (258829, 2)


In [82]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
verbose, epochs, batch_size = 0, 64, 64
print('Timesteps, number of features, n_outputs',n_timesteps,n_features,n_outputs)
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
_, accuracy_train = model.evaluate(train_x, train_y, batch_size=batch_size, verbose=0)
print('Accuracy on train dataset',accuracy_train)
_, accuracy_test = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
print('Accuracy on train dataset',accuracy_test)

Timesteps, number of features, n_outputs 41 1 2
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 202       
Total params: 51,102
Trainable params: 51,102
Non-trainable params: 0
_________________________________________________________________
None
Accuracy on train dataset 0.6775736808776855
Accuracy on train dataset 0.7831193804740906


*   Epochs 15, Train 62, Test 75
*   Epochs 50, Train, 67, Test 81
*   Epoch 64, Train 67, Test 78
*   Epoch 100, Train 75, Test 67

In [95]:
def evaluate_model(train_x, train_y, test_x, test_y):
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  verbose, epochs, batch_size = 0, 50, 64
  print('Timesteps, number of features, n_outputs',n_timesteps,n_features,n_outputs)
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
  _, accuracy_train = model.evaluate(train_x, train_y, batch_size=batch_size, verbose=0)
  print('Accuracy on train dataset',accuracy_train)
  _, accuracy_test = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=0)
  print('Accuracy on train dataset',accuracy_test)

In [96]:
# repeat experiment
repeats = 10
for r in range(repeats):
  print('Iteration number ',r,'Fitting the model')
  evaluate_model(train_x, train_y, test_x, test_y)	

Iteration number  0 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6604477763175964
Accuracy on train dataset 0.7342917323112488
Iteration number  1 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6713547706604004
Accuracy on train dataset 0.7478991746902466
Iteration number  2 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6713547706604004
Accuracy on train dataset 0.6801401972770691
Iteration number  3 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6600650548934937
Accuracy on train dataset 0.760873019695282
Iteration number  4 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6757558584213257
Accuracy on train dataset 0.7551819682121277
Iteration number  5 Fitting the model
Timesteps, number of features, n_outputs 41 1 2
Accuracy on train dataset 0.6711634